In [ ]:
def trace_mean_reverse(df, lag = 1, long_short_market = False, stock_outliers = False):
    r = df.pivot(index = 'd', columns = 'id', values = 'r')
    R = r.apply(lambda x: np.exp(x) - 1)
    
    def delta_mean(id_R):
        id_w_nonadj = (id_R - id_R.rolling(window=63, min_periods=10).mean()) / id_R.rolling(window=63, min_periods=10).std()
        return id_w_nonadj
    
    w_non_adj = R.apply(delta_mean)
    
    def weights_no_lag(day_R): 
        day_dR = day_R - day_R.mean()
        c = -1 / (day_dR.abs().sum()/2)
        return c * day_dR
    
    w = w_non_adj.apply(weights_no_lag, axis=1).shift(lag)
    w_long = pd.melt(w.reset_index(), id_vars=['d'], value_name='w')
    w_long['k'] = lag
    
    P_r = (w * R).sum(axis=1, skipna=False)
    mean_r = P_r.mean() * 252
    volatility = P_r.std() * math.sqrt(252)
    sharpe = mean_r / volatility
    
    # If we want extra information (performance of long, short and market for Q1), 
    # we will make additional operations:
    if long_short_market == True:
        L_r = (w[w > 0] * R).sum(axis=1)
        S_r = (w[w < 0] * R).sum(axis=1)
        M_r = R.apply(np.mean, axis=1)
        
        return [mean_r, volatility, sharpe], w_long, P_r, L_r, S_r, M_r
    
    elif stock_outliers == True:
        stock_returns = (w * R).sum(axis=0, skipna=True) 
        return [mean_r, volatility, sharpe], w_long, P_r, stock_returns
    
    else:
        return [mean_r, volatility, sharpe], w_long, P_r